In [2]:
import numpy as np
import pandas as pd 
from PIL import Image
import os

import torch
import torch.nn as nn
import torchvision.transforms as transform
from torch.utils.data import Dataset

from tqdm import tqdm
import random

#### set random seed

In [5]:
def setSeed(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

#### transform

In [4]:
test_tfm = transform.Compose([
    transform.Resize((128, 128)),
    transform.ToTensor()
])

train_tfm = transform.Compose([
    transform.Resize((128, 128)),
    transform.ToTensor()
])

#### dataset

In [6]:
class foodDataset(Dataset):

    def __init__(self, path, tfm=test_tfm):
        super(foodDataset).__init__()
        self.path = path
        self.tfm = tfm
        self.imgName = sorted([name for name in os.listdir(self.path) if name.endswith('.jpg')])
        self.imgPath = [os.path.join(self.path, name) for name in self.imgName]
     
    def __getitem__(self, idx):
        img = Image.open(self.imgPath[idx])
        img = self.tfm(img)
        try:
            label = int(self.imgName[idx].split('_')[0])
        except:
            label = -1
        return img, label
    
    def __len__(self, idx):
        return len(self.imgName)


#### model

In [9]:
class cnnBlock(nn.Module):

    def __init__(self, input_chann, output_channel, kernel_size=3, stride=1, padding=1):
        super(cnnBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(input_chann, output_channel, kernel_size=kernel_size, stride=stride, padding=padding),
            nn.BatchNorm2d(output_channel),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0)
        )

    def forward(self, x):
        return self.block(x)

In [12]:
class linearBlock(nn.Module):

    def __init__(self, input_dim, output_dim):
        self.block = nn.Sequential(
            nn.Linear(input_dim, output_dim),
            nn.ReLU()
        )
    
    def forward(self, x):
        return self.block(x)

In [13]:
class foodClassifier(nn.Module):
    
    def __init__(self, cnnLayers, linearLayers):
        super(foodClassifier, self).__init__()
        self.cnn = nn.Sequential(
            *[cnnBlock(cnnLayers[i-1], cnnLayers[i]) for i in range(1, len(cnnLayers))]
        )
        self.linear = nn.Sequential(
            *[linearBlock(linearLayers[i-1], linearLayers[i]) for i in range(1, len(linearLayers))]
        )
    
    def forward(self, x):
        x = self.cnn(x)
        x = x.flatten(start_dim=1)
        x = self.linear(x)
        return x
        

In [ ]:
config = {
    'learning_rate': 1e-4,
    'batch_size': 64,
    'cnnLayers': [3, 64, 128, 256, 512, 512],
    'linearLayers': [512*4*4, 1024, 512, 11],
    'gamma': 0.5,
    'step_size': 10,
    'weight_decay': 1e-2,
    'seed': 914122,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'epochNum': 400,
    'early_stop': 50
}